# file is present inside a folder called notebooks inside Project directory

In [135]:
import numpy as np
from sklearn.svm import OneClassSVM
import random


from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation, NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from scipy.spatial.distance import mahalanobis

import sys
sys.path.append('../')

from utils import get_all_vectors, filter_images
from feature_reduction.feature_reduction import reducer
from metric.distance import distance

In [199]:
model = 'sift'
k = 20
label = 'dorsal'
frt = 'pca'

In [200]:
#getting dorsal vectors and class
dorsal_paths = filter_images(label)
_, dorsal_vectors = get_all_vectors(model, f={'path': {'$in': dorsal_paths}})
dorsal_class = np.array([1] * len(dorsal_vectors))

In [201]:
_, palmar_vectors = get_all_vectors(model, f={'path': {'$nin': dorsal_paths}})
palmar_class = np.array([0] * len(palmar_vectors))

In [202]:
def predict_label(query_vector, dorsal_scaler, dorsal_pca, pca_dorsal_vectors, palmar_scaler, palmar_pca, pca_palmar_vectors):
    query_vector = query_vector.reshape(1,-1)
        
    #dorsal compute
    scaled_vec = dorsal_scaler.transform(query_vector)
    pca_query_vector_dorsal = dorsal_pca.transform(scaled_vec)
    mean_dorsal = np.mean(pca_dorsal_vectors, axis=0)
    dorsal_dist = mahalanobis(mean_dorsal, pca_query_vector_dorsal,np.linalg.inv(np.cov(pca_dorsal_vectors.T)))
    
    #palmar compute
    scaled_vec = palmar_scaler.transform(query_vector)
    pca_query_vector_palmar = palmar_pca.transform(scaled_vec)  
    mean_palmar = np.mean(pca_palmar_vectors, axis=0)
    palmar_dist = mahalanobis(mean_palmar, pca_query_vector_palmar, np.linalg.inv(np.cov(pca_palmar_vectors.T)))
    
    if dorsal_dist < palmar_dist: return 1
    return 0

In [203]:
def test():
    # Vectors for palmar and dorsal split into test and train
    vectors =  np.vstack((palmar_vectors, dorsal_vectors))
    labels = np.concatenate((palmar_class, dorsal_class))
    train_data, test_data, train_labels, test_labels = train_test_split(vectors, labels)
    
    #train do
    dorsal_train_data = train_data[np.where(train_labels == 1)[0]]
    palmar_train_data = train_data[np.where(train_labels == 0)[0]]
    
    #DORSAL
    pca_dorsal_vectors, _, _, dorsal_scaler, dorsal_pca = reducer(
    dorsal_train_data, 
    k, 
    frt,
    get_scaler_model = True
    )
    dorsal_inv = np.linalg.inv(np.cov(pca_dorsal_vectors))

    #palmar compute
    pca_palmar_vectors, _, _, palmar_scaler, palmar_pca = reducer(
    palmar_train_data, 
    k, 
    frt, 
    get_scaler_model = True
    )
    palmar_inv = np.linalg.inv(np.cov(pca_palmar_vectors))
    
    pred_labels = []
    for each in test_data:
        pred_labels.append(predict_label(
            each, dorsal_scaler, dorsal_pca, pca_dorsal_vectors, palmar_scaler, palmar_pca, pca_palmar_vectors
        ))

    score = accuracy_score(pred_labels, test_labels)
    return score

In [204]:
scores = []
tests = 10
for _ in range(tests):
    scores.append(test())

returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model
returning just the scaler and frt model


In [205]:
print(np.mean(scores))

0.7313559322033898
